- roll 미루는거
- cumsum 누적합
-각 구간에서 격차 가 큰 값이 적절한 안정 구간으로 된다
(낄이가 가장 긴거)




# [LAB 02] DBSCAN 성능평가

## 01.DBSCAN 성능평가 개요

### [1] 왜 DBSCAN 은 성능평가가 어려울까?
- 정답이 정해진 군집이 아님
- 군집수를 자동으로 결정해줌 (k means 처럼 정답 k 가 없음)
- 이상치를 허용함 (일부 데이터는 군집에 아예 속하지 않음, 즉 모든 데이터를 평가 대상으로 삼는 지표는 쓰지 못함)
- 비구형,비선형 군집
- K-MEANS 의 계열 지표와 전제 불일치 (실루엣, inertia 는 모든 점이 군집에 속하고, 군집은 구형이라는 전제가 있음)

### [2] 잘못된 접근
- 단일 성능 점수로 비교 --> DBSCAN 은 구간 기반 판단
- 실루엣 점수만으로 EPS 결정 --> 실루엣에는 NOISE 개념이 없어 보조 지표일 뿐임
- K 최적화 관점 적용 --> DBSCAN 은 K 를 찾아내는 것이 아닌 구조의 안정성을 확인해야함


### [3] DBSCAN 성능평가의 기본 관점
- 예측 성능에 초점 X
- 구조 안정성 --> EPS 가 변했을 떄 군집 구조가 흔들리는지?
- 이상치 의미--> NOISE 는 쓸모없는 데이터가 아닌 이상 패턴으로 인식
- 해석 가능성



### [4] 수치화 가능한 평가 지표 (보조적)

- 이상치 비율은 $ \text{Noise ratio} = \frac{n_{noise}}{n_{total}} $ 로 정의한다.
- eps 간 ARI

- eps 가 커지면 보통 noise 는 감소

### [5] eps grid 기반 성능평가 프레임

#### [5.1] eps 후보 설정

- k-distance elbow point --> 중심값 , eps 의 기준점 역할
- eps rlwns +- 20 -40% --> 이 구간 근저에서 구조의 변화를 봄

#### [5.2] eps grid 구성
- step : eps x 0.05
- 반복횟수 : 10-20회


#### [5.3] eps별 계산 지표 , 각 eps 마다 계산하는 지표

- 군집 수
- 이상치 비율
- ARI (Adjusted Rand Index):  
  두 군집 결과의 일치도를 무작위 일치 확률을 보정해  
  **-1 ~ 1 범위**로 측정하는 군집 안정성 비교 지표, 값이 높을수록 구조가 안정적


#### [5.4] 안정 구간의 정의
- 좋은 구간을 찾아내는 것이 목적
  
- ARI ≥ 0.9 유지 (구조 변화 거의 없음)
- 군집 수 급변 없음 (갑자기 쪼개지거나 합쳐지지 않음)
- 이상치 비율 완만 변화 (noise 가 서서히 변함)
- plateau 구간 존재 (지표들이 팽팽하게 유지되는 구간)



#### [5.5] 최종 eps 선택 원칙

- 단일 최적값 ❌
- 안정 구간 ⭕
- 구간 내 대표 eps 선택


#### [5.6] 목적 기반 평가 연결

- 이상치 탐지 목적 --> noise 를 많이 남기는 eps 가 적절
- 패턴 탐색 목적 --> 군집 구조가 명확한 eps
- 세분화·해석 목적 --> 군집수가 늘어나도 내부 구조를 자세히 볼 수 있음


## #02. 준비작업
### [1] 패키지 참조

In [2]:
from hossam import load_data, my_dpi
from pandas import DataFrame
from matplotlib import pyplot as plt
import seaborn as sb
import numpy as np

from sklearn.preprocessing import StandardScaler
from sklearn.cluster import DBSCAN
from sklearn.neighbors import NearestNeighbors

from scipy.spatial import ConvexHull

from kneed import KneeLocator

#성능평가지표
from sklearn.metrics import adjusted_rand_score




### [2] 데이터 가져오기

In [3]:
origin = load_data('game_usage')

게임 이용시간(time spent)과 레벨(game level)에 대한 가상 데이터


### [3] 데이터 전처리
- 종속변수 제거 + 데이터 표준화
  

In [4]:
scaler = StandardScaler()

df=DataFrame(scaler.fit_transform(origin),columns = origin.columns)
df.head()

,time spent,game level
0,-0.250733,1.474805
1,0.326494,0.606546
2,-0.611500,0.795456
3,0.470801,1.674613
4,-1.405187,-1.558652


## #02 min_samples 고정

## 실무 기준 규칙

- 일반 데이터: `min_samples = 5 ~ 10`
- 차원(d) 기준 경험적 선정 (아래 표 참고)

| 차원 수 | 권장 min_samples |
|-------|------------------|
| 2 ~ 3 | 3 ~ 6 |
| 5 ~ 10 | 8 ~ 15 |
| 10 이상 | 15 이상 |

> 본 실험에서는 **차원 수가 20**이므로  
> `min_samples = 3`으로 고정


In [5]:
min_samples=3

## #03. k 최근접 이웃을 통한 eps 값 도출

In [6]:
# k = min_samples 설정
k=min_samples


#각 점에 대해 k 번쨰 최근접 이웃 거리 계산
neighbors = NearestNeighbors (n_neighbors=k)
neighbors_fit = neighbors.fit(df)

#이떄 distance 는 각 데이터 포인트가 k 번째까지의 포인트와의 거리를 가까운 순서대로 리스트로 가지고 있음
distance ,indices = neighbors_fit.kneighbors(df)


#모든 점의 거리 값을 가까운 순서대로 정렬 (오름차순)
s_distance = np.sort(distance,axis=0)


#각 데이터 포인트로부터의 거리 추출
target = s_distance[:,k-1]  #k 번째 이웃은 k-1 번쨰 이웃에 존재 , 즉 가장 멀리 있는 값을 가져와서 


#엘보우 포인트 찾기
kl= KneeLocator(range(0,len(target)),target,curve='convex',direction='increasing')
eps=kl.elbow_y

#결과값
print(f'eps:{eps}')


eps:0.41251429498079606


## #03. eps 값을 기준으로 최적 eps 구간 찾기
### [1] eps 구간 잡기
eps 기준 -30% ~ +30% 구간을 5% 단위로 증가하는 구간 산정 (분석가 주관에 따라 구간과 증가값 설정 필요)

In [7]:
delta_ratio = 0.3
step_ratio=0.05

eps_min = eps*(1-delta_ratio)
eps_max=eps*(1+delta_ratio)
step=eps*step_ratio


eps_grid = np.arange(eps_min,eps_max+step,step)
eps_grid

array([0.28876001, 0.30938572, 0.33001144, 0.35063715, 0.37126287,
       0.39188858, 0.41251429, 0.43314001, 0.45376572, 0.47439144,
       0.49501715, 0.51564287, 0.53626858, 0.5568943 ])

### [2] eps 별 DBSCAN 결과 얻기

In [9]:
labels_dict={}

for eps in eps_grid:
  estimator = DBSCAN(eps=eps,min_samples=min_samples)
  labels_dict[eps] = estimator.fit_predict(df)


labels_dict

{np.float64(0.2887600064865572): array([-1,  0,  1, -1,  2,  1, -1,  3,  4,  3, -1,  8,  5,  6, -1,  5,  7,
         5,  1,  6,  8,  9, 10,  8,  5,  1,  1,  5,  4,  6,  0,  1,  4,  5,
         7,  1,  3,  4,  2,  1,  4,  7,  1, -1,  9,  0,  1,  3, -1, 11,  4,
        10, 11, -1, -1, -1,  2,  1,  9, -1, 11,  0, -1,  6,  3,  0,  3, -1,
         3,  6,  1, -1, -1,  4, -1, 10, -1,  0,  5,  6,  2,  8,  3,  1, -1,
        -1,  1,  3,  5, 10,  0, -1,  4, -1, -1, -1,  3,  1, 10,  4]),
 np.float64(0.309385721235597): array([ 0,  1,  2, -1,  3,  2,  4,  4,  5,  4, -1,  6,  7,  4, -1,  7,  8,
         7,  2,  4,  6,  8,  9,  6,  7,  2,  2,  7,  5,  4,  1,  2,  5,  7,
         8,  2,  4,  5,  3,  2,  5,  8,  2, 11,  8,  1,  2,  4,  1, 10,  5,
         9, 10, -1, -1,  2,  3,  2,  8, -1, 10,  1, 11,  4,  4,  1,  4, -1,
         4,  4,  2, -1, -1,  5, -1,  9,  0,  1,  7,  4,  3,  6,  4,  2, -1,
        11,  2,  4,  7,  9,  1, -1,  5, -1, -1,  0,  4,  2,  9,  5]),
 np.float64(0.3300114359846368): arra

### [3] eps 별 기본 지표

In [ ]:
rows = []


for eps in eps_grid:
  labels = 